#### Bert 모델 성능 평가
사용 데이터: od_uuid/2023/000c16dad0a74e3aa088fc8616b4b220

In [3]:
import pandas as pd
import os
import string
import folium
import geopy.distance

os.chdir('../')
data_path=os.getcwd()+'/data/od_uuid/2023'

#데이터셋 칼럼명 추가
column_name=['id','start_time','end_time','start_lat','start_lng','end_lat','end_lng','?1','?2','?3']

raw_data=pd.read_csv(data_path+'/000c16dad0a74e3aa088fc8616b4b220.csv')
data=raw_data.values.tolist()
dataframe=pd.DataFrame(data, columns=column_name)

In [4]:
# train/test 데이터 분할
datafilter1=dataframe['start_time'].str.contains("2023-10")
datafilter2=dataframe['start_time'].str.contains("2023-11")
datafilter3=dataframe['start_time'].str.contains("2023-12")

train_data=dataframe[~(datafilter1|datafilter2|datafilter3)]
test_data=dataframe[datafilter1|datafilter2|datafilter3]

그리드 생성을 위한 함수 선언

In [5]:
# Define a function to convert numbers into corresponding letter labels
def num_to_letter(num):
    '''
    num         : number that we have to convert
    '''
    return string.ascii_uppercase[num]

def generate_initial_grids():
        """초기 그리드 생성"""
        south, west, north, east = south_korea_bounds
        lat_step = (north - south) / grid_size
        lon_step = (east - west) / grid_size
        grid_queue = []

        for i in range(grid_size):
            for j in range(grid_size):
                grid_south = south + i * lat_step
                grid_north = south + (i + 1) * lat_step
                grid_west = west + j * lon_step
                grid_east = west + (j + 1) * lon_step
                grid_queue.append((grid_south, grid_west, grid_north, grid_east, 
                                   num_to_letter(i) + num_to_letter(j)))

        return grid_queue

# Create a function to get the grid label of the coordinate point
def get_grid_label(lat, lng, final_grids):
    '''
    lat         : latitude
    lng         : longitude
    final_grids : all cells and their minimum/maximum latitude/longitude
    '''
    for south, west, north, east, grid_label in final_grids:
        if south <= lat <= north and west <= lng <= east:
            return grid_label
    return None

def is_path_in_grid(south, west, north, east, path_points):
        """경로가 그리드 안에 있는지 확인"""
        return any(south <= lat <= north and west <= lng <= east for lat, lng in path_points)

def subdivide_grids(grid_queue, path_points):
    """그리드 분할"""
    final_grids = []

    while grid_queue:
        south, west, north, east, grid_label = grid_queue.pop(0)
        grid_size_km = min(geopy.distance.distance((south, west), (south, east)).km,
                           geopy.distance.distance((south, west), (north, west)).km)
            
        if grid_size_km > min_size_km and is_path_in_grid(south, west, north, east, path_points):
            mid_lat = (south + north) / 2
            mid_lon = (west + east) / 2
            grid_queue.append((south, west, mid_lat, mid_lon, grid_label + 'C'))
            grid_queue.append((mid_lat, west, north, mid_lon, grid_label + 'A'))
            grid_queue.append((south, mid_lon, mid_lat, east, grid_label + 'D'))
            grid_queue.append((mid_lat, mid_lon, north, east, grid_label + 'B'))
        else:
            final_grids.append((south, west, north, east, grid_label))

    return final_grids

# Approximate border coordinates of South Korea
south_korea_bounds = [33.10, 124.57, 38.60, 131]
min_size_km = 0.76
grid_size = 13



Train 데이터 시작/종료 위치 그리드 생성

In [6]:
start_points=train_data[['start_lat','start_lng']].values.tolist()
start_grid_queue = generate_initial_grids()
start_final_grids = subdivide_grids(start_grid_queue, start_points)

end_points=train_data[['end_lat','end_lng']].values.tolist()
end_grid_queue = generate_initial_grids()
end_final_grids = subdivide_grids(end_grid_queue, end_points)

train_data['start_grid']=train_data.apply(lambda row: get_grid_label(row['start_lat'], row['start_lng'], start_final_grids), axis=1)
train_data['end_grid']=train_data.apply(lambda row: get_grid_label(row['end_lat'], row['end_lng'], end_final_grids), axis=1)
train_data=train_data[['start_grid', 'end_grid']]

C:\Users\11\AppData\Local\Temp\ipykernel_16728\1362932771.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['start_grid']=train_data.apply(lambda row: get_grid_label(row['start_lat'], row['start_lng'], start_final_grids), axis=1)
C:\Users\11\AppData\Local\Temp\ipykernel_16728\1362932771.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['end_grid']=train_data.apply(lambda row: get_grid_label(row['end_lat'], row['end_lng'], end_final_grids), axis=1)


Test 데이터 시작/종료 위치 그리드 생성

In [7]:
start_points=test_data[['start_lat','start_lng']].values.tolist()
start_grid_queue = generate_initial_grids()
start_final_grids = subdivide_grids(start_grid_queue, start_points)

end_points=test_data[['end_lat','end_lng']].values.tolist()
end_grid_queue = generate_initial_grids()
end_final_grids = subdivide_grids(end_grid_queue, end_points)

test_data['start_grid']=test_data.apply(lambda row: get_grid_label(row['start_lat'], row['start_lng'], start_final_grids), axis=1)
test_data['end_grid']=test_data.apply(lambda row: get_grid_label(row['end_lat'], row['end_lng'], end_final_grids), axis=1)
test_data=test_data[['start_grid', 'end_grid']]


C:\Users\11\AppData\Local\Temp\ipykernel_16728\3916890138.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['start_grid']=test_data.apply(lambda row: get_grid_label(row['start_lat'], row['start_lng'], start_final_grids), axis=1)
C:\Users\11\AppData\Local\Temp\ipykernel_16728\3916890138.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['end_grid']=test_data.apply(lambda row: get_grid_label(row['end_lat'], row['end_lng'], end_final_grids), axis=1)


In [8]:
test_data
train_data

,start_grid,end_grid
0,KGCACADB,KGCACBAD
1,KGCACBAD,KGCACBBB
2,KGCABCBD,KGCACBBC
3,KGCACBAD,KGACCDDC
4,KGCABACC,KGCACBBB
...,...,...
877,KFBBCDAD,KFBBCDAD
878,KFBBCDAB,KFBBCDAC
879,KFBBCDAD,KFBBCDAD
880,KFBBCDAD,KFBBCDAD


#### Bert 모델 테스팅

In [24]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Encode the target variable End_grid
label_encoder = LabelEncoder()
all_labels = list(train_data['end_grid']) + list(test_data['end_grid'])
label_encoder.fit(all_labels)

train_data['end_grid_encoded'] = label_encoder.transform(train_data['end_grid'])
test_data['end_grid_encoded'] = label_encoder.transform(test_data['end_grid'])

# Extract the features and target
train_X = train_data['start_grid'].tolist()
train_y = train_data['end_grid_encoded'].tolist()
test_X = test_data['start_grid'].tolist()
test_y = test_data['end_grid_encoded'].tolist()



# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom Dataset class for our data
class GridPathDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Parameters
MAX_LEN = 8
BATCH_SIZE = 8

# Create dataset
train_dataset = GridPathDataset(train_X, train_y, tokenizer, MAX_LEN)
test_dataset = GridPathDataset(test_X, test_y, tokenizer, MAX_LEN)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model training setup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

print("Training complete.")

# Evaluation
model.eval()
val_targets = []
val_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)

        val_targets.extend(labels.cpu().numpy())
        val_predictions.extend(preds.cpu().numpy())

# Calculate performance metrics
accuracy = accuracy_score(val_targets, val_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(val_targets, val_predictions, average='weighted')

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation Precision: {precision:.4f}')
print(f'Validation Recall: {recall:.4f}')
print(f'Validation F1 Score: {f1:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\11\anaconda3\envs\wedrive\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss: 4.224711894989014
Epoch: 0, Loss: 4.0392045974731445
Epoch: 0, Loss: 3.7075023651123047
Epoch: 0, Loss: 4.052983283996582
Epoch: 0, Loss: 3.53875732421875
Epoch: 0, Loss: 3.332005023956299
Epoch: 0, Loss: 3.4924941062927246
Epoch: 0, Loss: 2.554784059524536
Epoch: 0, Loss: 3.19659161567688
Epoch: 0, Loss: 2.8790833950042725
Epoch: 0, Loss: 3.2511305809020996
Epoch: 0, Loss: 3.752807855606079
Epoch: 0, Loss: 3.4340105056762695
Epoch: 0, Loss: 2.4506208896636963
Epoch: 0, Loss: 3.4043328762054443
Epoch: 0, Loss: 2.3500301837921143
Epoch: 0, Loss: 4.26141357421875
Epoch: 0, Loss: 0.9881395697593689
Epoch: 0, Loss: 3.017796754837036
Epoch: 0, Loss: 1.9529849290847778
Epoch: 0, Loss: 1.924493670463562
Epoch: 0, Loss: 1.442360281944275
Epoch: 0, Loss: 3.299684762954712
Epoch: 0, Loss: 2.1568853855133057
Epoch: 0, Loss: 1.3945114612579346
Epoch: 0, Loss: 3.3202497959136963
Epoch: 0, Loss: 2.2900595664978027
Epoch: 0, Loss: 2.692734718322754
Epoch: 0, Loss: 1.6136741638183594
E

c:\Users\11\anaconda3\envs\wedrive\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### 파라미터 & 결과
1. 파라미터
MAX_LEN = 128,
BATCH_SIZE = 16,
EPOCHS = 3

Validation Accuracy: 0.5278   
Validation Precision: 0.3182   
Validation Recall: 0.5278   
Validation F1 Score: 0.3904   

2. 파라미터
MAX_LEN = 128,
BATCH_SIZE = 8,
EPOCHS = 5

Validation Accuracy: 0.5278   
Validation Precision: 0.3182   
Validation Recall: 0.5278   
Validation F1 Score: 0.3904   

3. 파라미터
MAX_LEN = 64,
BATCH_SIZE = 8,
EPOCHS = 3

Validation Accuracy: 0.5301   
Validation Precision: 0.3191   
Validation Recall: 0.5301   
Validation F1 Score: 0.3919   

4. 파라미터
MAX_LEN = 8,
BATCH_SIZE = 8,
EPOCHS = 3

Validation Accuracy: 0.5440    
Validation Precision: 0.3494   
Validation Recall: 0.5440   
Validation F1 Score: 0.4234    